In [4]:
clear()

In [5]:
#Set up
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn import tree

In [6]:
data = pd.read_csv( "HW3_training.csv")
data

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,CASH_OUT,94865.94,41338.00,0.00,0.00,94865.94,0
1,CASH_IN,318248.86,2980.00,321228.86,336116.40,17867.54,0
2,PAYMENT,17546.87,0.00,0.00,0.00,0.00,0
3,CASH_OUT,126683.04,11162.85,0.00,2519453.70,2646136.74,0
4,CASH_OUT,150054.00,356.00,0.00,0.00,150054.00,0
...,...,...,...,...,...,...,...
628966,PAYMENT,23295.72,7234.00,0.00,0.00,0.00,0
628967,CASH_OUT,274781.27,126263.81,0.00,603764.98,878546.26,0
628968,PAYMENT,9874.46,217132.07,207257.61,0.00,0.00,0
628969,PAYMENT,11457.28,0.00,0.00,0.00,0.00,0


In [7]:
Fraud =  data[data['isFraud'] == 1]
Fraud

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
722,TRANSFER,479892.04,479892.04,0.0,0.00,0.00,1
1088,CASH_OUT,435166.65,435166.65,0.0,449215.85,884382.50,1
4533,CASH_OUT,475368.94,475368.94,0.0,1348026.73,1823395.67,1
4826,CASH_OUT,4094.07,4094.07,0.0,296534.20,300628.27,1
5745,CASH_OUT,400157.86,400157.86,0.0,0.00,400157.86,1
...,...,...,...,...,...,...,...
626882,TRANSFER,195351.76,195351.76,0.0,0.00,0.00,1
627952,TRANSFER,22877.00,22877.00,0.0,0.00,0.00,1
628328,CASH_OUT,1917497.63,1917497.63,0.0,173507.14,2091004.76,1
628775,CASH_OUT,865948.49,865948.49,0.0,0.00,865948.49,1


In [8]:
X=data[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']]
Y=data['isFraud']
#Normalize numerical Data
data['oldbalanceOrg']=(data['oldbalanceOrg'] - data['oldbalanceOrg'].min()) / (data['oldbalanceOrg'].max() - data['oldbalanceOrg'].min()) 
data['newbalanceOrig']=(data['newbalanceOrig'] - data['newbalanceOrig'].min()) / (data['newbalanceOrig'].max() - data['newbalanceOrig'].min()) 
data['oldbalanceDest']=(data['oldbalanceDest'] - data['oldbalanceDest'].min()) / (data['oldbalanceDest'].max() - data['oldbalanceDest'].min()) 
data['newbalanceDest']=(data['newbalanceDest'] - data['newbalanceDest'].min()) / (data['newbalanceDest'].max() - data['newbalanceDest'].min()) 
data['amount']=(data['amount'] - data['amount'].min()) / (data['amount'].max() - data['amount'].min())

# split data randomly
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.4, random_state = 213)

In [9]:
# train a linear regression

Linearreg = LinearRegression().fit(train_x, train_y)
print(Linearreg.intercept_, Linearreg.coef_, Linearreg.score(train_x, train_y))

#accuracy
lin_pred = Linearreg.predict(test_x)
linearAccuracy = 1-np.sum(np.abs(test_y-lin_pred))/len(test_y)
print(linearAccuracy)

0.00034694362368664026 [ 2.18362079e-08  9.27181590e-08 -9.18279006e-08  9.05453530e-09
 -9.21253760e-09] 0.13806709887450785
0.9932026081552804


In [10]:
# train a logistic model
Logitreg = LogisticRegression(solver='newton-cg').fit(train_x, train_y)
print(Logitreg.intercept_, Logitreg.coef_, Logitreg.score(train_x, train_y))

#accuracy
pred = Logitreg.predict(test_x)
Logitregaccuracy=np.mean(pred == test_y)
print('Accuracy= ', Logitregaccuracy)

[-1.37252545e-10] [[-4.52958656e-06 -4.66424146e-06 -6.13872460e-06 -3.80860490e-06
  -5.78103883e-06]] 0.9985770386504921
Accuracy=  0.998509473784625


/Users/liaohang/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/liaohang/Library/Python/3.9/lib/python/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


In [11]:
# train a tree
aTreeClassifier = tree.DecisionTreeClassifier(max_depth=5)
aTreeClassifier = aTreeClassifier.fit(train_x, train_y)
print(aTreeClassifier.score(train_x, train_y))

#accuracy
pred = aTreeClassifier.predict(test_x)
Treeaccuracy=np.mean(pred == test_y)
print('Accuracy= ', Treeaccuracy)

0.998966564383039
Accuracy=  0.9988155285008486


In [65]:
# train a KNN
K = [1,2,3,4,5,6,7,8,9,10]
for k in K:
    KNN = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
    KNN.fit(train_x, train_y)
    print("When K is ",k, ", the score is ",KNN.score(train_x, train_y))

When K is  1 , the score is  1.0
When K is  2 , the score is  0.9993905379694845
When K is  3 , the score is  0.9993269419315177
When K is  4 , the score is  0.9992818947379578
When K is  5 , the score is  0.9992447970491438
When K is  6 , the score is  0.9992262482047368
When K is  7 , the score is  0.9991918003508381
When K is  8 , the score is  0.999181201011177
When K is  9 , the score is  0.9991653020016853
When K is  10 , the score is  0.9991706016715158


In [12]:
#accuracy
#Final output KKN when k = 1
KNN = KNeighborsClassifier(n_neighbors=1, metric = 'euclidean')
KNN.fit(train_x, train_y)

pred = KNN.predict(test_x)
KNN1accuracy=np.mean(pred == test_y)
print('K =', 1, ', Accuracy =', KNN1accuracy)

K = 1 , Accuracy = 0.9986883369304699


In [13]:
####PREDICT####
new_data = pd.read_csv( "HW3_test_input.csv")
X_new=new_data[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']]
new_data

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,CASH_OUT,119240.82,0.00,0.00,188216.08,296238.92
1,PAYMENT,2799.78,48932.00,46132.22,0.00,0.00
2,CASH_OUT,134843.00,21278.00,0.00,0.00,134843.00
3,TRANSFER,631622.55,0.00,0.00,2708960.94,3793415.70
4,CASH_OUT,135241.51,0.00,0.00,2448960.30,2584201.81
...,...,...,...,...,...,...
209939,CASH_OUT,51420.99,45283.00,0.00,717858.34,769279.33
209940,CASH_IN,99882.73,2732875.35,2832758.08,1057750.11,957867.38
209941,CASH_IN,42374.82,256.00,42630.82,3755635.14,3713260.33
209942,PAYMENT,20558.23,0.00,0.00,0.00,0.00


From the score of the four predictions, we could know that when k = 1, KKN has the highest score, so we should choose KKN as the prediction and make k = 1;

In [14]:
if max(linearAccuracy, Logitregaccuracy, KNN1accuracy, Treeaccuracy)==linearAccuracy:
    bestModel="Linear model"
elif max(linearAccuracy, Logitregaccuracy, KNN1accuracy, Treeaccuracy)==Logitregaccuracy:
    bestModel="Logistic Model"
elif max(linearAccuracy, Logitregaccuracy, KNN1accuracy, Treeaccuracy)==KNN1accuracy:
    bestModel="KNN Model"
else: bestModel="Tree Model"
print('Best Model is', bestModel)

Best Model is Tree Model


In [15]:
#Tree prediction
y_TreePredict=aTreeClassifier.predict(X_new)
np.savetxt('HW3_test_output.csv', y_TreePredict, delimiter=',')

In [16]:
new=pd.read_csv( "HW3_test_output.csv")
new.columns = ['isFraud']
new=new[new['isFraud'] == 1]
new
## The following transactions are the fraud as predicted

,isFraud
4403,1.0
4672,1.0
5896,1.0
6848,1.0
8890,1.0
...,...
204744,1.0
207010,1.0
207113,1.0
207444,1.0


From the score of the four predictions, we could know that when k = 1, KKN has the highest score, so we should choose KKN as the prediction and make k = 1;

In [17]:
#Final output KKN when k = 1
KNN = KNeighborsClassifier(n_neighbors=1, metric = 'euclidean')
KNN.fit(train_x, train_y)
y_KNNPredict=KNN.predict(X_new)
np.savetxt('HW3_test_output.csv', y_KNNPredict, delimiter=',')

In [18]:
new=pd.read_csv( "HW3_test_output.csv")
new.columns = ['isFraud']
new=new[new['isFraud'] == 1]
new
## The following transactions are the fraud as predicted

,isFraud
1825,1.0
3930,1.0
4040,1.0
4403,1.0
4672,1.0
...,...
207444,1.0
207553,1.0
207633,1.0
208059,1.0
